# Fine tune StableLM-2-Zephyr-1.6B model using Axolotl framework

How to install dependencies (in HPC environment):

- load Python and cuDNN modules
- create a Python venv and activate it
- install dependencies from requirements.txt (e.g. torch)
- install Axolotl from git clone (pip won't work, see [this issue](https://github.com/OpenAccess-AI-Collective/axolotl/issues/945)):

```
git clone git@github.com:OpenAccess-AI-Collective/axolotl.git
cd axolotl
pip install -e '.[flash-attn,deepspeed]'
```


In [1]:
# Check if GPU is available
import torch
print('GPU available?', torch.cuda.is_available())
print('BF16 is supported?', torch.cuda.is_bf16_supported())

GPU available? True
BF16 is supported? True


In [2]:
# set model name etc.

MODEL_NAME = "stabilityai/stablelm-2-zephyr-1_6b"
MODEL_SHORT_NAME = MODEL_NAME.split('/')[-1]

In [3]:
# Load and prepare fine-tuning dataset

import json
import glob
import random

random.seed(42)  # for deterministic sampling of test set

train_files = glob.glob("../../llm-dataset/*-train.jsonl")
test_files = glob.glob("../../llm-dataset/*-test.jsonl")

KEEP_FIELDS = {
    'dc.contributor.author',
    'dc.date.issued',
    'dc.identifier.isbn',
    'dc.language.iso',
    'dc.publisher',
    'dc.relation.eissn',
    'dc.title'    
}
MAX_TEXT_LENGTH = 3072
EVAL_SIZE = 32  # how many documents to evaluate (i.e. calculate loss) on during fine-tuning
SYSTEM_PROMPT = "You are a skilled librarian specialized in meticulous cataloguing of digital documents."
INSTRUCTION = "Extract metadata from this document. Return as JSON."

def preprocess_sample(sample):
    # subset & JSON encode the ground truth
    subset = {key: val
              for key, val in sample["ground_truth"].items()
              if key in KEEP_FIELDS}
    if 'dc.date.issued' in subset:
        # keep only the year
        subset['dc.date.issued'] = subset['dc.date.issued'][:4]
    if 'dc.identifier.isbn' in subset:
        # normalize ISBN by stripping dashes
        subset['dc.identifier.isbn'] = [isbn.replace('-', '') for isbn in subset['dc.identifier.isbn']]
    output = json.dumps(subset)
    input_ = sample["text"][:MAX_TEXT_LENGTH]
    # ShareGPT format
    conversations = [
        {'from': 'system', 'value': SYSTEM_PROMPT},
        {'from': 'user', 'value': INSTRUCTION + "\n\n" + input_},
        {'from': 'gpt', 'value': output}
    ]
    return {"conversations": conversations}

def dataset_to_records(files):
    records = []
    for filename in files:
        with open(filename) as infile:
            for line in infile:
                sample = json.loads(line)
                records.append(preprocess_sample(sample))
    return records

def write_jsonl(records, filename):
    with open(filename, "w") as outfile:
        for record in records:
            json.dump(record, outfile)
            outfile.write("\n")

train_recs = dataset_to_records(train_files)
random.shuffle(train_recs)
write_jsonl(train_recs, "train.jsonl")
print(f"Wrote {len(train_recs)} train records")

test_recs = dataset_to_records(test_files)
write_jsonl(test_recs, "test.jsonl")
print(f"Wrote {len(test_recs)} test records")

eval_recs = random.sample(test_recs, EVAL_SIZE)
write_jsonl(eval_recs, "eval.jsonl")
print(f"Wrote {len(eval_recs)} eval records")

Wrote 556 train records
Wrote 167 test records
Wrote 32 eval records


In [4]:
# Create Axolotl configuration file

CONFIG_FILE = f"config-{MODEL_SHORT_NAME}.yml"


CONFIG = f"""
base_model: {MODEL_NAME}
model_type: AutoModelForCausalLM
tokenizer_type: AutoTokenizer
trust_remote_code: true

load_in_8bit: true
load_in_4bit: false
strict: false

datasets:
  - path: train.jsonl
    ds_type: json
    split: train
    type: sharegpt
    conversation: chatml

test_datasets:
  - path: eval.jsonl
    ds_type: json
    split: train
    type: sharegpt
    conversation: chatml

output_dir: ./out-{MODEL_SHORT_NAME}

chat_template: chatml

adapter: lora
lora_r: 16
lora_alpha: 32
lora_dropout: 0.05
lora_target_linear: true

sequence_len: 4096
sample_packing: false
eval_sample_packing: false
pad_to_sequence_len: true

wandb_project:
wandb_entity:
wandb_watch:
wandb_name:
wandb_log_model:

gradient_accumulation_steps: 4
micro_batch_size: 4
eval_batch_size: 4
num_epochs: 5
optimizer: adamw_bnb_8bit
lr_scheduler: cosine
learning_rate: 0.0002

train_on_inputs: false
group_by_length: false
bf16: true
fp16: false
tf32: false

gradient_checkpointing: true  # true: saves VRAM but is slower to train
early_stopping_patience:
resume_from_checkpoint:
local_rank:
logging_steps: 1
xformers_attention:
flash_attention: false  # true gives RuntimeError: FlashAttention only support fp16 and bf16 data type

warmup_steps: 10
evals_per_epoch: 2
eval_table_size:
eval_table_max_new_tokens: 128
saves_per_epoch: 1
debug:
weight_decay: 0.0
fsdp:
fsdp_config:
special_tokens:

""".strip()

with open(CONFIG_FILE, 'w') as outfile:
    print(CONFIG, file=outfile)

In [5]:
%%time

!venv/bin/accelerate launch -m axolotl.cli.train {CONFIG_FILE}

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
[2024-03-28 14:57:36,271] [INFO] [datasets.<module>:58] [PID:2806237] PyTorch version 2.2.0 available.
[2024-03-28 14:57:37,768] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-03-28 14:57:39,420] [WARNING] [axolotl.utils.config.models.input.hint_trust_remote_code:283] [PID:2806237] [RANK:0] `trust_remote_code` is set to true. Please make sure that you reviewed the remote code/model.
[2024-03-28 14:57:39,595] [INFO] [axolotl.normalize_config:178] [PID:2806237] [RANK:0] GPU memory usage baseline: 0.000GB (+0.818GB misc)
[2024-03-28 14:57:39,597] [WARNING] [acc

# Use the fine-tuned model

In [6]:
import torch
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

qlora_model = f"./out-{MODEL_SHORT_NAME}"
base_model = MODEL_NAME
tokenizer = AutoTokenizer.from_pretrained(qlora_model, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(base_model, device_map=0, trust_remote_code=True, torch_dtype="auto", attn_implementation="flash_attention_2",)
model = PeftModel.from_pretrained(model, qlora_model)



/wrk-vakka/users/oisuomin/git/FinGreyLit/experiments/axolotl-finetune-llm/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# push the LoRA model (PEFT adapter) and modified tokenizer (added ChatML template) to HF Hub

hub_model_id = f"NatLibFi/{MODEL_SHORT_NAME}-meteor"
model.push_to_hub(hub_model_id)
tokenizer.push_to_hub(hub_model_id)

adapter_model.safetensors: 100%|██████████| 30.3M/30.3M [00:01<00:00, 17.8MB/s]


CommitInfo(commit_url='https://huggingface.co/NatLibFi/stablelm-2-zephyr-1_6b-meteor/commit/ee9fa701f874cb5819f0d95a792124a3df20ac09', commit_message='Upload tokenizer', commit_description='', oid='ee9fa701f874cb5819f0d95a792124a3df20ac09', pr_url=None, pr_revision=None, pr_num=None)

In [8]:
# merge the LoRA into the base model for inference

model = model.merge_and_unload()

In [9]:
def generate(messages):
    input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True)
    output_ids = model.generate(
        torch.as_tensor(input_ids).cuda(),
        #input_ids,
        max_new_tokens=512,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    output_ids = output_ids[0][len(input_ids[0]):]
    return tokenizer.decode(output_ids, skip_special_tokens=True).strip()


In [10]:
%%time

import json

with open(f'test-records-{MODEL_SHORT_NAME}.jsonl', 'w') as outfile:
    for rec in test_recs:
        messages = [
            {"role": msg["from"], "content": msg["value"]}
            for msg in rec["conversations"]
            if msg["from"] != "gpt"
        ]
        response = generate(messages)

        ground_truth = rec['conversations'][-1]["value"]

        print(100 * "-")
        print("Ground Truth:")
        print(ground_truth)
        print("Prediction:")
        print(response)

        ground_truth = json.loads(ground_truth)

        try:
            prediction = json.loads(response)
        except json.JSONDecodeError:
            prediction = {}
        
        # rowid is set to unknown as we've lost it somewhere along the way...
        record = {"ground_truth": ground_truth, "prediction": prediction, "rowid": "unknown"}
        json.dump(record, outfile)
        outfile.write("\n")

----------------------------------------------------------------------------------------------------
Ground Truth:
{"dc.title": "Poliisikoulutuksen vaikuttavuusarviointi 2021 : vuosina 2018-2019 valmistuneiden poliisien ty\u00f6llisyys ja arviot koulutuksen ty\u00f6el\u00e4m\u00e4vastaavuudesta", "dc.contributor.author": ["Vuorensyrj\u00e4, Matti"], "dc.date.issued": "2021", "dc.identifier.isbn": ["9789518153866"], "dc.language.iso": "fin", "dc.publisher": ["Poliisiammattikorkeakoulu"]}
Prediction:
{"dc.title": "Poliisikoulutuksen vaikuttavuusarviointi 2021 sisus POAMK.pdf", "dc.contributor.author": ["Vuorensyrj\u00e4, Matti"], "dc.date.issued": "2021", "dc.identifier.isbn": ["9789518153866"], "dc.language.iso": "fin", "dc.publisher": ["Poliisiammattikorkeakoulu"]}
----------------------------------------------------------------------------------------------------
Ground Truth:
{"dc.title": "FAQ : Taiteen digitaaliset toimintaymp\u00e4rist\u00f6t", "dc.date.issued": "2018", "dc.identif

In [11]:
# Analyze the statistics of the extracted metadata and save to file

import sys
sys.path.append('..')

from eval import MetadataEvaluator

evaluator = MetadataEvaluator(f'test-records-{MODEL_SHORT_NAME}.jsonl')
results = evaluator.evaluate_records() #prediction_records[:9])
# Use only the fields that Meteor extracts
fields = [
        "dc.contributor.author",
        "dc.date.issued",
        "dc.identifier.isbn",
        "dc.language.iso",
        "dc.publisher",
        "dc.relation.eissn",
        "dc.title",
    ]
statistics_filename = '../results-axolotl-fine-tune-' + MODEL_SHORT_NAME + '.md'
evaluator.save_md(results, statistics_filename, fields)

In [12]:
# Save the merged model to a directory (along with tokenizer)

merged_model_dir = f"merged-{MODEL_SHORT_NAME}"
model.save_pretrained(merged_model_dir)
tokenizer.save_pretrained(merged_model_dir)

('merged-stablelm-2-zephyr-1_6b/tokenizer_config.json',
 'merged-stablelm-2-zephyr-1_6b/special_tokens_map.json',
 'merged-stablelm-2-zephyr-1_6b/vocab.json',
 'merged-stablelm-2-zephyr-1_6b/merges.txt',
 'merged-stablelm-2-zephyr-1_6b/added_tokens.json',
 'merged-stablelm-2-zephyr-1_6b/tokenizer.json')

In [2]:
%%time
# convert the merged model to GGUF using llama.cpp tools (installed separately)

LLAMA_CPP_PATH = "../../../llama.cpp"

!{LLAMA_CPP_PATH}/venv/bin/python {LLAMA_CPP_PATH}/convert-hf-to-gguf.py {merged_model_dir} --outfile {MODEL_SHORT_NAME}-meteor-f16.gguf

/bin/bash: {LLAMA_CPP_PATH}/venv/bin/python: No such file or directory
CPU times: user 1.15 ms, sys: 4.2 ms, total: 5.35 ms
Wall time: 117 ms


In [1]:
%%time
# Quantize the F16 GGUF model to the 4+ bit Q4_K_M format
QTYPE = "Q4_K_M"

!{LLAMA_CPP_PATH}/quantize {MODEL_SHORT_NAME}-meteor-f16.gguf {MODEL_SHORT_NAME}-meteor-{QTYPE}.gguf {QTYPE}

/bin/bash: {LLAMA_CPP_PATH}/quantize: No such file or directory
CPU times: user 5.36 ms, sys: 8.66 ms, total: 14 ms
Wall time: 151 ms


In [ ]:
# Upload the quantized GGUF file to HF Hub

FINAL_MODEL_NAME = f"NatLibFi/{MODEL_SHORT_NAME}-meteor-GGUF"

!{LLAMA_CPP_PATH}/venv/bin/huggingface-cli upload {FINAL_MODEL_NAME} {MODEL_SHORT_NAME}-meteor-{QTYPE}.gguf